# nodel analysis with clinical improvement

In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import correlation
import numpy as np

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

from statistical_analysis.ttest import load_clinical

node_edges = 246
obs_name1 = 'base'
obs_name2 = '030d'
filename = 'fc_r_BN246.txt'
intensity_thres = 0.3

c_names = ['CRST_A','CRST_B', 'CRST_C', 'CRST_TOTAL' ]

for c_name in c_names:
    clinical_v1 = load_clinical(subjects, obs_name1, c_name)
    clinical_v2 = load_clinical(subjects, obs_name2, c_name)
    clinical_delta = clinical_v1 - clinical_v2

    for j in range(node_edges):

        all_edges_delta = []
        for subject in subjects:
            observation1 = subject.get_observation(obs_name1)
            observation2 = subject.get_observation(obs_name2)
            
            fc1 = np.loadtxt(observation1.bold.build_path(filename))
            fc2 = np.loadtxt(observation2.bold.build_path(filename))
            
            # node_edge 0-245
            edges1 = fc1[j]
 
            edges1 = np.absolute(edges1)

            edges1[edges1<intensity_thres] = 0

            edges2 = fc2[j]

            edges1 = np.absolute(edges1)

            edges1[edges1<intensity_thres] = 0

            edges_delta = edges1 - edges2
            all_edges_delta.append(edges_delta)
        
        all_edges_delta = np.array(all_edges_delta).T
        
        rs = []
        for edge in all_edges_delta:
            r,p = correlation.correlation(clinical_delta, edge)
            if p < 0.05:
                rs.append(r)

        rs = np.abs(rs)
        sum_r = np.sum(rs)
        
        print('{}: ROI{}, sum_r={}'.format(c_name, j+1, sum_r))

# nodal trend analysis

In [ ]:
# sequence based
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

threshold_value = 0.3 
#out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta' 

metrics = ['degree']
from sklearn.linear_model import LinearRegression
from scipy import stats

for metric in metrics:
    subjects_values = []
    obs_x = []
    columns = []
    for subject in subjects:
        x = 1
        observations = subject.get_all_observation()
        values = []
        for obs in observations:
            if obs.name != 'ncnc':
                value = obs.bold.get_nodal_metric(metric, mat_name='brant_post/wholenet_network.mat', 
                                                threshold_type='intensity', 
                                                threshold_value=threshold_value)
                columns.append(obs.name)
                subjects_values.append(value)
                #print(obs.name)
                obs_x.append(x)
                x += 1
    obs_x = np.array(obs_x)
    subjects_values = np.array(subjects_values)
    subjects_values = subjects_values.T
    i = 1
    for node_values in subjects_values:
        slope, intercept, r_value, p_value, std_err = stats.linregress(obs_x, node_values)
        print('Node,{},coef,{}, p,{}'.format(i, slope, p_value))
        """
        reg = LinearRegression().fit(obs_x.reshape(-1, 1), node_values)
        print('Node:{} coef:{}'.format(i, reg.coef_))
        y_pred = reg.predict(obs_x.reshape(-1, 1))
        """
        
        #plt.scatter(obs_x, node_values,  color='black')
        #plt.plot(obs_x, obs_x*slope+intercept, color='blue', linewidth=3)

        #plt.xticks(())
        #plt.yticks(())

        #plt.show()
        i += 1
    


In [ ]:
#base on postoperative time
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')


threshold_value = 0.3 
#out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta' 

metrics = ['degree']
from sklearn.linear_model import LinearRegression
from scipy import stats

for metric in metrics:
    subjects_values = []
    obs_x = []
    columns = []
    for subject in subjects:
        x = 1
        observations = subject.get_all_observation()
        values = []
        for obs in observations:
            if obs.name != 'ncnc':
                value = obs.bold.get_nodal_metric(metric, mat_name='brant_post/wholenet_network.mat', 
                                                threshold_type='intensity', 
                                                threshold_value=threshold_value)
                columns.append(obs.name)
                subjects_values.append(value)
                #print(obs.name)
                if obs.name =='base':
                    obs_x.append(0)
                else:
                    obs_x.append(int(obs.name[:-1]))
    obs_x = np.array(obs_x)
    subjects_values = np.array(subjects_values)
    subjects_values = subjects_values.T
    i = 1
    for node_values in subjects_values:
        slope, intercept, r_value, p_value, std_err = stats.linregress(obs_x, node_values)
        print('Node,{},coef,{}, p,{}'.format(i, slope, p_value))
        """
        reg = LinearRegression().fit(obs_x.reshape(-1, 1), node_values)
        print('Node:{} coef:{}'.format(i, reg.coef_))
        y_pred = reg.predict(obs_x.reshape(-1, 1))
        """
        
        #plt.scatter(obs_x, node_values,  color='black')
        #plt.plot(obs_x, obs_x*slope+intercept, color='blue', linewidth=3)

        #plt.xticks(())
        #plt.yticks(())

        #plt.show()
        i += 1
    


# specific node data show

In [ ]:
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd
from statistical_analysis import correlation

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')


threshold_value = 0.3 
out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta'

#metrics = ['degree','global_efficiency', 'transitivity']

metrics = ['degree']

c_names = ['handtremor','CRSTA_total','CRST b_total','CRST C','CRST TOTAL']
_, ax = plt.subplots(figsize=(float(9), float(6)))

with open(r'D:\Desktop\temppppp\nodal\a.txt', 'w') as f:
    for i in range(246):
        for metric in metrics:
            for c_name in c_names:
                subjects_values = []
                clinical_values = []
                for subject in subjects:
                    observations = subject.get_all_observation()
                    for obs in observations:
                        if obs.name != 'ncnc' and obs.name != '001d'and obs.name != '007d':
                            value = obs.bold.get_nodal_metric(metric, mat_name='brant_post/wholenet_network.mat', 
                                                            threshold_type='intensity', 
                                                            threshold_value=threshold_value)
                            subjects_values.append(value)
                            clinical_values.append(obs.args[c_name])

                subjects_values = np.array(subjects_values).T

                subjects_values = subjects_values[i]
                r, p = correlation.correlation(subjects_values, clinical_values, show=False)
                if p<0.05/246:
                    r, p = correlation.correlation(subjects_values, clinical_values, show=True)
                    print('EOI,{},-{}-,{}, r,{}, p,{}'.format(i+1, metric, c_name, r, p), file=f)
